# सेमांटिक कर्नेल

इस कोड उदाहरण में, आप [सेमांटिक कर्नेल](https://aka.ms/ai-agents-beginners/semantic-kernel) एआई फ्रेमवर्क का उपयोग करके एक बुनियादी एजेंट बनाएंगे।

इस उदाहरण का उद्देश्य आपको उन चरणों को दिखाना है, जिन्हें हम बाद में विभिन्न एजेंटिक पैटर्न को लागू करते समय अतिरिक्त कोड उदाहरणों में उपयोग करेंगे।


## आवश्यक पायथन पैकेज आयात करें


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## क्लाइंट बनाना

इस उदाहरण में, हम [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) का उपयोग LLM तक पहुंचने के लिए करेंगे।

`ai_model_id` को `gpt-4o-mini` के रूप में परिभाषित किया गया है। GitHub Models मार्केटप्लेस में उपलब्ध किसी अन्य मॉडल पर स्विच करके विभिन्न परिणाम देखने का प्रयास करें।

`Azure Inference SDK` का उपयोग करने के लिए, जो GitHub Models के `base_url` के लिए उपयोग किया जाता है, हम Semantic Kernel के भीतर `OpenAIChatCompletion` कनेक्टर का उपयोग करेंगे। Semantic Kernel को अन्य मॉडल प्रदाताओं के लिए उपयोग करने के लिए अन्य [उपलब्ध कनेक्टर्स](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) भी हैं।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## एजेंट बनाना

नीचे हम `TravelAgent` नामक एजेंट बनाते हैं।

इस उदाहरण में, हम बहुत सरल निर्देशों का उपयोग कर रहे हैं। आप इन निर्देशों को बदल सकते हैं ताकि देख सकें कि एजेंट अलग-अलग तरीके से कैसे प्रतिक्रिया करता है।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## एजेंट चलाना

अब हम एजेंट को चलाने के लिए `ChatHistoryAgentThread` प्रकार का एक थ्रेड परिभाषित कर सकते हैं। किसी भी आवश्यक सिस्टम संदेश एजेंट के invoke_stream के `messages` कीवर्ड आर्गुमेंट में प्रदान किए जाते हैं।

इनके परिभाषित होने के बाद, हम एक `user_inputs` बनाते हैं, जो उपयोगकर्ता द्वारा एजेंट को भेजा जाने वाला संदेश होगा। इस मामले में, हमने इस संदेश को `Plan me a sunny vacation` पर सेट किया है।

आप इस संदेश को बदलने के लिए स्वतंत्र हैं ताकि देख सकें कि एजेंट अलग-अलग तरीके से कैसे प्रतिक्रिया देता है।


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
